# Prepocesamiento
1. Cargar data

In [51]:
import os

In [52]:
#Path de los datos 
#data_path = r'C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\test_txt' 
data_path = r'D:\U\7. Septimo\RI\ProyectoRI\data\training_txt'

In [53]:
#Crear un vector de documentos
#Abrimos cada archivo y lo leemos
documents = []
for filename in os.listdir(data_path):
    if filename.endswith('.txt'):
        path = os.path.join(data_path, filename)
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
            documents.append((filename, content))
len(documents)

7769

 2. StopWords y Stemming

In [54]:
from nltk.stem import SnowballStemmer 

In [55]:
#Stemmer se usa para reducir las palabras a su raíz
stemmer = SnowballStemmer('english')

In [56]:
#Path de las stopwords
stopwords_path = r"D:\U\7. Septimo\RI\ProyectoRI\data\stopwords.txt"

In [57]:
#Abrimos el archivo y leemos las stopwords
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines())

3. Tokenización

In [64]:
#Función para tokenizar y stemmizar
#tokenizar: dividir el texto en palabras
import nltk
import string
import re  # Asegúrate de importar el módulo 're' para expresiones regulares


In [65]:
#Definimos una función que normaliza el texto con todos los requisitos necesarios:
#conviertimos en minúsculas
#eliminamos los signos de puntuación
#tokenizamos
#aplicamos stemming
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    processed_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(processed_tokens)


4. Aplicación a todos los documentos

In [66]:
#Preprocesamos los documentos
preprocessed_documents = [(filename, preprocess_text(content)) for filename, content in documents]


In [67]:
len(preprocessed_documents)

7769

In [68]:
'''
#Guardamos los documentos preprocesados en un directorio 
preprocessed_data_path = r'D:\U\7. Septimo\RI\ProyectoRI\data\preprocessed_txt'
os.makedirs(preprocessed_data_path, exist_ok=True)
for filename, content in preprocessed_documents:
    with open(os.path.join(preprocessed_data_path, filename), 'w', encoding='utf-8') as file:
        file.write(content)
'''

Transformacion de tupla a lista
tupla = ()
lista = []

In [69]:
folder_path = 'D:\\U\\7. Septimo\\RI\\ProyectoRI\\data\\cats.txt'

In [70]:
lines = []
with open(folder_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [71]:
def build_inverted_index(lines):
    index = {}
    for line in lines:
        # Separar la ruta del documento y los términos
        parts = line.strip().split()
        document = parts[0]
        terms = parts[1:]
        
        for term in terms:
            if term in index:
                index[term].append(document)
            else:
                index[term] = [document]
    return index

# Construir el índice invertido
inverted_index = build_inverted_index(lines)

In [72]:
preprocessed_documents_list = [doc[0] if isinstance(doc, tuple) else doc for doc in preprocessed_documents]

In [73]:

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


//

In [74]:
# Extraer los textos preprocesados
texts = [content for _, content in preprocessed_documents]

# Vectorizacion

In [75]:
# Vectorización con Bag of Words (BoW)
bow_vectorizer = CountVectorizer()
bow_matrix = bow_vectorizer.fit_transform(texts)

In [76]:
# Vectorización con TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

In [77]:
# Verificar la forma de las matrices
print("BoW matrix shape:", bow_matrix.shape)
print("TF-IDF matrix shape:", tfidf_matrix.shape)

BoW matrix shape: (7769, 21411)
TF-IDF matrix shape: (7769, 21411)


# 2.5 Motor de Busqueda

In [78]:
def find_similar_documents(matrix, index, top_n=10):
    cosine_similarities = cosine_similarity(matrix[index:index+1], matrix).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-top_n-1:-1]
    return related_docs_indices

# Índice del documento para el cual deseas encontrar documentos relevantes
doc_index = 5  # Puedes cambiar este valor

# Encontrar documentos similares usando BoW
similar_docs_bow = find_similar_documents(bow_matrix, doc_index)

# Encontrar documentos similares usando TF-IDF
similar_docs_tfidf = find_similar_documents(tfidf_matrix, doc_index)

print(f"Documentos similares al documento {doc_index} usando BoW: {similar_docs_bow}")
print(f"Documentos similares al documento {doc_index} usando TF-IDF: {similar_docs_tfidf}")


Documentos similares al documento 5 usando BoW: [   5 2367 6949 2264 5991 6083 5889  643 3486 2284]
Documentos similares al documento 5 usando TF-IDF: [   5 2367 2284 2264 5991 3486 7125 5889 3632 6396]


//